In [ ]:
'amortizacion_min_LB',
'cuota_max_LB_SMLV',

'numero_obligaciones_activasdif',#
'porcentaje_utilizacion',#
'quanto_mod',#
'valor_cuotas_codeudores_smlv',#
'valor_utilizado_smlv',#

'num_lib_solicitadas',#
'prom_monto_novado',#
'prom_n_cuotas',#

In [ ]:
var_finales = [
#
#


'num_no_aceptado',#
    
'dias_desde_ult_tx_ahr',#

'pasv_antig_total',#
'pasv_saldo_ca_1mes_atras',#

'sum_tx_linea_ahr',#
'num_tx_ult_1mes_ahr',#
    
]

# Construcción variables Activo

In [1]:
import pandas as pd 
import numpy as np 

pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

from datetime import datetime, timedelta 
from dateutil import relativedelta
import s3fs
import pyarrow.parquet as pq
import gc

In [2]:
fs = s3fs.S3FileSystem()

### Se leen los archivos del activo

In [3]:
path = 's3://adl-refined-dev-popular/data_parquet/productos/activo/productos_activo_sarc_M'

#['201801','201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
str_dates  = ['201904','201905','201906','201907','201908','201909','201910','201911','201912',
            '202001','202002','202003','202004','202005','202006']

str_dates2 = ['201901','201902','201903']

In [4]:
def read_file(date, tipo):
    
    if tipo == 'pq':
        path = 's3://adl-refined-dev-popular/data_parquet/productos/activo/productos_activo_sarc_M'
        input_path = path+date
        input = input_path
        dataset = pq.ParquetDataset(input, filesystem=fs)
        table = dataset.read()
        table = table.to_pandas()
        table.columns = table.columns.str.lower()
        
        del dataset
        
    elif tipo in ['csv','txt']:
        path = 's3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M'
        input_path = path+date+'.'+tipo
        types = {'id_cliente': str, 'nrooblig': str}
        table = pd.read_csv(input_path, sep='|', encoding='utf-8', dtype=types)
        table.columns = table.columns.str.lower()
    else:
        table = []
        
    return table

In [5]:
for ix, date in enumerate(str_dates):
    print('Archivo:',date)
    
    data_temp = read_file(date, 'pq')
    data_temp['periodo'] = date
    
    if ix == 0:
        data_act = data_temp.copy()
    else:
        data_act = pd.concat([data_act,data_temp],ignore_index=True)
        
    del data_temp
    gc.collect()

Archivo: 201904
Archivo: 201905
Archivo: 201906
Archivo: 201907
Archivo: 201908
Archivo: 201909
Archivo: 201910
Archivo: 201911
Archivo: 201912
Archivo: 202001
Archivo: 202002
Archivo: 202003
Archivo: 202004
Archivo: 202005
Archivo: 202006


In [6]:
## Casting
data_act = data_act[data_act['id_cliente'].notnull()]
data_act['id_cliente'] = data_act['id_cliente'].astype('int64')
data_act['lineacredi'] = data_act['lineacredi'].astype('int64')
#Se eliminan duplicados
data_act=data_act.drop_duplicates(subset=['id_cliente','nrooblig','periodo'],keep='first')

In [7]:
# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M201901.csv",delimiter="|",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns] 
# prueba['fecha']='201901'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M201902.csv",delimiter="|",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns] 
# prueba['fecha']='201902'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M201903.csv",delimiter="|",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns] 
# prueba['fecha']='201903'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

In [8]:
# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M202002.csv",delimiter=";",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns]
# prueba['fecha']='202002'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M202003.csv",delimiter="|",dtype={'id_cliente':object,'nrooblig':object})
# prueba.columns = [x.lower() for x in prueba.columns]
# prueba['fecha']='202003'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M202004.csv",delimiter=";",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns]
# prueba['fecha']='202004'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

# prueba=pd.read_csv("s3://adl-refined-dev-popular/data_orig/productos/activo/productos_activo_sarc_M202005.csv",delimiter=";",dtype={'ID_CLIENTE':object,'NROOBLIG':object})
# prueba.columns = [x.lower() for x in prueba.columns]
# prueba['fecha']='202005'
# data_act = pd.concat([data_act,prueba],ignore_index=True)

In [9]:
data_act['periodo'].value_counts().sort_index()

201904    566417
201905    565432
201906    561726
201907    560519
201908    562850
201909    562428
201910    567632
201911    569916
201912    571888
202001    568820
202002    572425
202003    569105
202004    563224
202005    561272
202006    560625
Name: periodo, dtype: int64

In [10]:
data_act.shape

(8484279, 21)

In [11]:
# Porcentaje cruce:

str_dates = list(pd.Series(data_act['periodo'].unique()).sort_values())
dates = list(pd.Series(data_act['periodo'].unique()).sort_values())[2:]

print('Para TC expres')

for ix,date in enumerate(dates):
    temp_id = list(data_act.loc[data_act['periodo'] == date,'id_cliente'].unique())
    temp_id2 = list(data_act.loc[data_act['periodo'] == str_dates[ix],'id_cliente'].unique())
    print('Fecha',date,'vs',str_dates[ix],':',round(len(np.intersect1d(temp_id,temp_id2))/len(temp_id)*100,2),'%')
    del temp_id, temp_id2

Para TC expres
Fecha 201906 vs 201904 : 96.75 %
Fecha 201907 vs 201905 : 95.2 %
Fecha 201908 vs 201906 : 93.27 %
Fecha 201909 vs 201907 : 93.98 %
Fecha 201910 vs 201908 : 91.62 %
Fecha 201911 vs 201909 : 91.28 %
Fecha 201912 vs 201910 : 92.63 %
Fecha 202001 vs 201911 : 91.06 %
Fecha 202002 vs 201912 : 90.06 %
Fecha 202003 vs 202001 : 91.48 %
Fecha 202004 vs 202002 : 96.95 %
Fecha 202005 vs 202003 : 96.52 %
Fecha 202006 vs 202004 : 96.1 %


In [12]:
data_act.dtypes

tipo_ident        object
sk_cliente        object
id_cliente         int64
tipo_producto     object
lineacredi         int64
nrooblig          object
cod_ofic          object
plazo              int64
cuota_paga         int64
fecdes            object
cupo_aprob       float32
fecven            object
vlr_desemb       float32
sal_capita       float32
vlr_cuota        float32
tasint_cte       float32
tasint_mor       float32
dias_morak         int64
estado            object
calif_cart        object
periodo           object
dtype: object

In [13]:
# Porcentaje cruce:

str_dates = list(pd.Series(data_act['periodo'].unique()).sort_values())
dates = list(pd.Series(data_act['periodo'].unique()).sort_values())[2:]

print('Para TC expres')

for ix,date in enumerate(dates):
    temp_id = list(data_act.loc[(data_act['periodo'] == date) & (data_act['lineacredi']==62),'id_cliente'].unique())
    temp_id2 = list(data_act.loc[(data_act['periodo'] == str_dates[ix]) & (data_act['lineacredi']==62),'id_cliente'].unique())
    print('Fecha',date,'vs',str_dates[ix],':',round(len(np.intersect1d(temp_id,temp_id2))/len(temp_id)*100,2),'%')
    del temp_id, temp_id2

Para TC expres
Fecha 201906 vs 201904 : 93.67 %
Fecha 201907 vs 201905 : 90.91 %
Fecha 201908 vs 201906 : 85.15 %
Fecha 201909 vs 201907 : 86.22 %
Fecha 201910 vs 201908 : 75.31 %
Fecha 201911 vs 201909 : 72.83 %
Fecha 201912 vs 201910 : 78.31 %
Fecha 202001 vs 201911 : 72.26 %
Fecha 202002 vs 201912 : 70.23 %
Fecha 202003 vs 202001 : 75.72 %
Fecha 202004 vs 202002 : 93.56 %
Fecha 202005 vs 202003 : 88.64 %
Fecha 202006 vs 202004 : 88.67 %


In [14]:
# Porcentaje cruce:

str_dates = list(pd.Series(data_act['periodo'].unique()).sort_values())
dates = list(pd.Series(data_act['periodo'].unique()).sort_values())[2:]

print('Para TC tradicional')

for ix,date in enumerate(dates):
    temp_id = list(data_act.loc[(data_act['periodo'] == date) & (data_act['lineacredi']==61),'id_cliente'].unique())
    temp_id2 = list(data_act.loc[(data_act['periodo'] == str_dates[ix]) & (data_act['lineacredi']==61),'id_cliente'].unique())
    print('Fecha',date,'vs',str_dates[ix],':',round(len(np.intersect1d(temp_id,temp_id2))/len(temp_id)*100,2),'%')
    del temp_id, temp_id2

Para TC tradicional
Fecha 201906 vs 201904 : 93.88 %
Fecha 201907 vs 201905 : 93.37 %
Fecha 201908 vs 201906 : 93.12 %
Fecha 201909 vs 201907 : 93.45 %
Fecha 201910 vs 201908 : 93.15 %
Fecha 201911 vs 201909 : 93.25 %
Fecha 201912 vs 201910 : 93.66 %
Fecha 202001 vs 201911 : 93.1 %
Fecha 202002 vs 201912 : 91.71 %
Fecha 202003 vs 202001 : 92.03 %
Fecha 202004 vs 202002 : 94.26 %
Fecha 202005 vs 202003 : 94.46 %
Fecha 202006 vs 202004 : 91.9 %


In [15]:
## Validación de duplicados a nivel 'id_cliente','nrooblig','periodo'

In [16]:
data_act=data_act.sort_values(['id_cliente','nrooblig','periodo'])

In [17]:
## Se eliminan id_cliente vacios o negativos

In [18]:
data_act=data_act[(data_act['id_cliente']>0)]

In [19]:
data_act.shape

(8484279, 21)

In [20]:
data_act['periodo'].value_counts().sort_index()

201904    566417
201905    565432
201906    561726
201907    560519
201908    562850
201909    562428
201910    567632
201911    569916
201912    571888
202001    568820
202002    572425
202003    569105
202004    563224
202005    561272
202006    560625
Name: periodo, dtype: int64

In [21]:
data_act.shape

(8484279, 21)

In [22]:
##Se castean la variables id:cliente y fecha

In [23]:
data_act['id_cliente']=data_act['id_cliente'].astype(int)

In [24]:
data_act['fecha_activo']=data_act['periodo'].astype(int)

In [25]:
##Creación varibles SMLV

In [26]:
data_act['ano_po'] = data_act['periodo'].str[0:4].astype(int)

In [27]:
#Se asigna el salario minimo mensual vigente de 2018 y 2019
data_act['SMLV'] = np.where(data_act['ano_po'] == 2018, 781242,
                               np.where(data_act['ano_po'] == 2019, 828116,
                                        np.where(data_act['ano_po'] == 2020, 877803,
                                                 np.nan)))

### Creación base a nivel id_cliente, fecha

In [28]:
data_act_cli_fec=pd.DataFrame(data_act.groupby(['id_cliente','periodo'])[['SMLV','fecha_activo']].max())

In [29]:
data_act_cli_fec.shape

(7512810, 2)

### Marca de producto incluyendo tdc expresss y tradicional

In [30]:
data_act['tipo_prod']=np.where((data_act['lineacredi']==61) & (data_act['tipo_producto']=='TC'),'TC_T',
                                   np.where((data_act['lineacredi']==62) & (data_act['tipo_producto']=='TC'),'TC_X',data_act['tipo_producto']))

### Variables  número de aperturas

In [31]:
#Cambio de formato de variables fecha desembolso y fecha observación

In [32]:
data_act['fecha_desembolso']=pd.to_datetime(pd.to_datetime(data_act['fecdes']).dt.date)

In [33]:
data_act['fecha_obs']=pd.to_datetime(pd.to_datetime(data_act['periodo'].astype(str).str[0:4]+'/'+data_act['periodo'].astype(str).str[4:6]+'/01').dt.date)

In [34]:
#Marca de apertura realizada en los ultimos x meses
data_act['apertura_1meses']=np.where((data_act['fecha_desembolso']<=data_act['fecha_obs']) & (data_act['fecha_desembolso']>=data_act['fecha_obs'] - timedelta(30)),1,0)
data_act['apertura_3meses']=np.where((data_act['fecha_desembolso']<=data_act['fecha_obs']) & (data_act['fecha_desembolso']>=data_act['fecha_obs'] - timedelta(90)),1,0)
data_act['apertura_6meses']=np.where((data_act['fecha_desembolso']<=data_act['fecha_obs']) & (data_act['fecha_desembolso']>=data_act['fecha_obs'] - timedelta(180)),1,0)
data_act['apertura_12meses']=np.where((data_act['fecha_desembolso']<=data_act['fecha_obs']) & (data_act['fecha_desembolso']>=data_act['fecha_obs'] - timedelta(365)),1,0)
    

In [35]:
#Marca de producto incluyendo tdc expresss y tradicional

In [36]:
data_act_cli_fec.columns

Index(['SMLV', 'fecha_activo'], dtype='object')

In [37]:
# Numero de aperturas total ultimos 1, 3, 6, 9 y 12 meses por cartera

antiguedad=['1','3','6','12']
cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']
def num_aper_tot(antiguedad,cartera,tipo_prod):
    data_act_cli_fec['num_aper_ultim_'+antiguedad+'meses_'+cartera] = data_act[data_act[tipo_prod]==cartera].groupby(['id_cliente','periodo'])[['apertura_'+antiguedad+'meses']].sum()
    data_act_cli_fec.fillna({'num_aper_ultim_'+antiguedad+'meses_'+cartera : 0}, inplace = True)
    
for ant in antiguedad:
    for car in cartera:
        num_aper_tot(ant,car,'tipo_producto')
        
for ant in antiguedad:
    for car in cartera1:
        num_aper_tot(ant,car,'tipo_prod')

In [38]:
# Numero de aperturas total ultimos 1,3, 6, 9 y 12 meses total

def num_aper_tot_tot(antiguedad):
    data_act_cli_fec['num_aper_ultim_'+antiguedad+'meses_tot'] = data_act.groupby(['id_cliente','periodo'])[['apertura_'+antiguedad+'meses']].sum()
    data_act_cli_fec.fillna({'num_aper_ultim_'+antiguedad+'meses_tot' : 0}, inplace = True)
    
for ant in antiguedad:
    num_aper_tot_tot(ant)

### Variables meses desde la ultima apertura

In [39]:
#Marca edad credito: meses entre la fecha apertura y la fecha observacion
data_act['edad_credito_meses']=(data_act['periodo'].str[0:4].astype('int')-data_act['fecha_desembolso'].astype(str).str[0:4].astype('int'))*12+(data_act['periodo'].str[4:7].astype('int')-data_act['fecha_desembolso'].astype(str).str[4:7].astype('int'))


In [40]:
#Funcion que calcula los meses desde la ultima apertura por cartera
cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']
def meses_ult_aper(cartera,tipo_prod):
    data_act_cli_fec['meses_ultim_aper_'+cartera] = data_act[data_act[tipo_prod]==cartera].groupby(['id_cliente','periodo'])[['edad_credito_meses']].min()
    
for car in cartera:
    meses_ult_aper(car,'tipo_producto')
        
for car in cartera1:
    meses_ult_aper(car,'tipo_prod')

In [41]:
#calculo de los meses desde la ultima apertura total
data_act_cli_fec['meses_ultim_aper_tot'] = data_act.groupby(['id_cliente','periodo'])[['edad_credito_meses']].min()
       

### Variables antiguedad del cliente

In [42]:
#Variable antiguedad (meses desde la primera apertura) con todas las carteras  con y sin libranza
cartera_conlib=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera_sinlib=['TC','NV','CH','CE','LE','ME' ,'SO','CC']

data_act_cli_fec['antiguedad_tot_conlibranza'] = data_act[data_act['tipo_producto'].isin(cartera_conlib)].groupby(['id_cliente','periodo'])[['edad_credito_meses']].max()
data_act_cli_fec['antiguedad_tot_sinlibranza'] = data_act[data_act['tipo_producto'].isin(cartera_sinlib)].groupby(['id_cliente','periodo'])[['edad_credito_meses']].max()


In [43]:
#Se calcula los meses desde la primera apertura en cualquier cartera  con y sin libranza

In [44]:
#Variable antiguedad (meses desde la primera apertura) en TDC
data_act_cli_fec['antiguedad_TDC_T'] = data_act[(data_act['lineacredi']==61) & (data_act['tipo_producto']=='TC')].groupby(['id_cliente','periodo'])[['edad_credito_meses']].max()
data_act_cli_fec['antiguedad_TDC_X'] = data_act[(data_act['lineacredi']==62) & (data_act['tipo_producto']=='TC')].groupby(['id_cliente','periodo'])[['edad_credito_meses']].max()


### Variables número obligaciones vigentes

In [45]:
#Marca de cuentas vigentes usando la variable estado cartera
data_act['cuenta_vigente'] = np.where(data_act['estado']=='ACTIVO',1,0)

In [46]:
# Numero de obligaciones vigentes de diferentes carteras a la fecha del reporte

cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']

def num_vigentes(cartera,tipo_producto):
    
    data_act_cli_fec['num_obl_vig_'+cartera] = data_act[data_act[tipo_producto]==cartera].groupby(['id_cliente','periodo'])[['cuenta_vigente']].sum()  
    data_act_cli_fec.fillna({'num_obl_vig_'+cartera : 0}, inplace = True)

for car in cartera:
    num_vigentes(car,'tipo_producto')

for car in cartera1:
    num_vigentes(car,'tipo_prod')

In [47]:
# Numero de obligaciones vigentes todas las carteras a la fecha del reporte
data_act_cli_fec['num_obl_vig_tot'] = data_act.groupby(['id_cliente','periodo'])[['cuenta_vigente']].sum()  
data_act_cli_fec.fillna({'num_obl_vig_tot': 0}, inplace = True)

# Variables número y porcentaje de obligaciones por altura de mora

In [48]:
#Marca de mora
data_act['altura_mora']=np.where(data_act['dias_morak']==0,'aldia',
np.where((data_act['dias_morak']>=1) & (data_act['dias_morak']<=30),'mora1_30',
np.where((data_act['dias_morak']>=31) & (data_act['dias_morak']<=60),'mora31_60',
np.where((data_act['dias_morak']>=61) & (data_act['dias_morak']<=90),'mora61_90',
np.where((data_act['dias_morak']>=91) & (data_act['dias_morak']<=120),'mora91_120',
np.where((data_act['dias_morak']>=121) ,'mora121_mas','nan'))))))

In [49]:
#Número y porcentaje de obligaciones en las diferentes alturas de mora del total de obligaciones activas diferentes carteras
moras = ['aldia','mora1_30','mora31_60','mora61_90','mora91_120','mora121_mas']

def porcentaje_mora_cartera(cartera, mora,tipo_prod):
    
    """
    Se crea la variable numero de obligaciones de la cartera en la mora definida, 
    y a partir de ésta se crea la variable porcentaje, con el total de obligaciones
    vigentes de la cartera
    """
    
    data_act_cli_fec['num_obl_' + mora + '_' + cartera] = data_act[(data_act[tipo_prod] == cartera) & 
                                                                     (data_act['altura_mora'] == mora)].groupby(['id_cliente','periodo'])[['id_cliente']].count()
    
    data_act_cli_fec.fillna({'num_obl_' + mora + '_' + cartera : 0}, inplace = True)
    
    data_act_cli_fec['porc_obl_' + mora + '_' + cartera] = np.where(data_act_cli_fec['num_obl_vig_' + cartera] == 0,
                                                                  np.nan, 
                                                                  (data_act_cli_fec['num_obl_' + mora + '_' + cartera]/data_act_cli_fec['num_obl_vig_' + cartera]))   
    
for car in cartera:
    for mor in moras:
        porcentaje_mora_cartera(car, mor,'tipo_producto')

for car in cartera1:
    for mor in moras:
        porcentaje_mora_cartera(car, mor,'tipo_prod')

In [50]:
#Número y porcentaje de obligaciones en las diferentes alturas de mora del total de obligaciones activas todas las carteras
def porcentaje_mora_cartera_tot(mora,cartera):
    data_act_cli_fec['num_obl_' + mora + '_' + cartera] = data_act[(data_act['altura_mora'] == mora)].groupby(['id_cliente','periodo'])[['id_cliente']].count()
    
    data_act_cli_fec.fillna({'num_obl_' + mora + '_' + cartera : 0}, inplace = True)
    
    data_act_cli_fec['porc_obl_' + mora + '_' + cartera] = np.where(data_act_cli_fec['num_obl_vig_' + cartera] == 0,np.nan, 
                                                                  (data_act_cli_fec['num_obl_' + mora + '_' + cartera]/data_act_cli_fec['num_obl_vig_' + cartera]))   


for mor in moras:
    porcentaje_mora_cartera_tot(mor,'tot')


In [51]:
import gc
gc.collect()

20

### Variables saldo, cupo, cuota al punto de observación

In [52]:
#Saldo, cupo y cuota diferentes carteras: suma total, maximo y promedio
cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']

def saldo_cupo_cuota_cartera(cartera,tipo_prod):

    data_act_cli_fec['saldo_tot_'+cartera] = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].sum()
    data_act_cli_fec['cupo_tot_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].sum()
    data_act_cli_fec['cuota_tot_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].sum()

    data_act_cli_fec['saldo_max_'+cartera] = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].max()
    data_act_cli_fec['cupo_max_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].max()
    data_act_cli_fec['cuota_max_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].max()

    data_act_cli_fec['saldo_prom_'+cartera] = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].mean()
    data_act_cli_fec['cupo_prom_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].mean()
    data_act_cli_fec['cuota_prom_'+cartera]  = data_act[(data_act[tipo_prod] ==cartera) & (data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].mean()
    
for car in cartera:
    saldo_cupo_cuota_cartera(car,'tipo_producto') 

for car in cartera1:
    saldo_cupo_cuota_cartera(car,'tipo_prod') 


In [53]:
#Saldo, cupo y cuota todas las carteras: suma total, maximo y promedio
cartera='tot'
data_act_cli_fec['saldo_tot_'+cartera] = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].sum()
data_act_cli_fec['cupo_tot_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].sum()
data_act_cli_fec['cuota_tot_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].sum()

data_act_cli_fec['saldo_max_'+cartera] = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].max()
data_act_cli_fec['cupo_max_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].max()
data_act_cli_fec['cuota_max_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].max()

data_act_cli_fec['saldo_prom_'+cartera] = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['sal_capita']].mean()
data_act_cli_fec['cupo_prom_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_desemb']].mean()
data_act_cli_fec['cuota_prom_'+cartera]  = data_act[(data_act['cuenta_vigente'] ==1)].groupby(['id_cliente','periodo'])[['vlr_cuota']].mean()
 

### Variables Utilizacion y Amortizacion

In [54]:
#Variable utilizacion a nivel obligación
data_act['utilizacion']=np.where((data_act['vlr_desemb']>0) & (data_act['sal_capita']>= 0),data_act['sal_capita']/data_act['vlr_desemb'],
np.where((data_act['vlr_desemb']>0) & np.isnan(data_act['sal_capita']),0,np.nan))

#Variable amortizacion a nivel obligación
data_act['amortizacion']=np.where((data_act['vlr_desemb']> 0) & (data_act['sal_capita'] >= 0),(data_act['vlr_desemb']-data_act['sal_capita'])/data_act['vlr_desemb'],
np.where((data_act['vlr_desemb'] > 0) & np.isnan(data_act['sal_capita']),1,np.nan))

In [55]:
#Utilización promedio, minima y máxima de todas las tarjetas de crédito activas a nivel cliente
cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']

def util(cartera,tipo_prod):
    data_act_cli_fec['utilizacion_prom_'+cartera] = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['utilizacion']].mean()

    data_act_cli_fec['utilizacion_min_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['utilizacion']].min()

    data_act_cli_fec['utilizacion_max_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['utilizacion']].max()

for car in cartera1:
    util(car,'tipo_prod')

#Para TC
util('TC','tipo_producto')    

In [56]:
#Amortización promedio, minima y máxima por cartera, sins tener en cuenta tarjetas de crédito
def amort(cartera,tipo_prod):
    data_act_cli_fec['amortizacion_prom_'+cartera] = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['amortizacion']].mean()

    data_act_cli_fec['amortizacion_min_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['amortizacion']].min()

    data_act_cli_fec['amortizacion_max_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      (data_act[tipo_prod] ==cartera)].groupby(['id_cliente','periodo'])[['amortizacion']].max()

cartera2=['LB','NV','CH','CE','LE','ME' ,'SO','CC']
for car in cartera2:
    amort(car,'tipo_producto')


In [57]:
#Amortizacion total sin incluir TC
cartera='tot'
data_act_cli_fec['amortizacion_prom_'+cartera] = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                    ~(data_act['tipo_producto']=='TC')].groupby(['id_cliente','periodo'])[['amortizacion']].mean()

data_act_cli_fec['amortizacion_min_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                     ~(data_act['tipo_producto'] =='TC')].groupby(['id_cliente','periodo'])[['amortizacion']].min()

data_act_cli_fec['amortizacion_max_'+cartera]  = data_act[ (data_act['cuenta_vigente'] ==1) & 
                                                      ~(data_act['tipo_producto'] =='TC')].groupby(['id_cliente','periodo'])[['amortizacion']].max()


### Variables Mora máxima a la fecha por cartera

In [58]:
#mora maxima a la fecha
cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
cartera1=['TC_X','TC_T']

def mora_max(cartera,tipo_producto):
    data_act_cli_fec['mora_max'+cartera] = data_act[ (data_act['cuenta_vigente'] ==1) & (data_act[tipo_producto]==cartera)].groupby(['id_cliente','periodo'])[['dias_morak']].max()

cartera=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC']
for car in cartera:
    mora_max(car,'tipo_producto')    
    
for car in cartera1:
    mora_max(car,'tipo_prod') 
    
#mora maxima total    
data_act_cli_fec['mora_max_tot'] = data_act[ (data_act['cuenta_vigente'] ==1) ].groupby(['id_cliente','periodo'])[['dias_morak']].max()

### Creación variables saldo, cupo, cuota sobre SMLV

In [59]:
variables_saldo_cupo_cuota=['saldo_tot_LB',
'saldo_max_LB',
'saldo_prom_LB',
'saldo_tot_TC',
'saldo_max_TC',
'saldo_prom_TC',
'saldo_tot_NV',
'saldo_max_NV',
'saldo_prom_NV',
'saldo_tot_CH',
'saldo_max_CH',
'saldo_prom_CH',
'saldo_tot_CE',
'saldo_max_CE',
'saldo_prom_CE',
'saldo_tot_LE',
'saldo_max_LE',
'saldo_prom_LE',
'saldo_tot_ME',
'saldo_max_ME',
'saldo_prom_ME',
'saldo_tot_SO',
'saldo_max_SO',
'saldo_prom_SO',
'saldo_tot_CC',
'saldo_max_CC',
'saldo_prom_CC',
'saldo_tot_TC_X',
'saldo_max_TC_X',
'saldo_prom_TC_X',
'saldo_tot_TC_T',
'saldo_max_TC_T',
'saldo_prom_TC_T',
'saldo_tot_tot',
'saldo_max_tot',
'saldo_prom_tot',
'cupo_tot_LB',
'cupo_max_LB',
'cupo_prom_LB',
'cupo_tot_TC',
'cupo_max_TC',
'cupo_prom_TC',
'cupo_tot_NV',
'cupo_max_NV',
'cupo_prom_NV',
'cupo_tot_CH',
'cupo_max_CH',
'cupo_prom_CH',
'cupo_tot_CE',
'cupo_max_CE',
'cupo_prom_CE',
'cupo_tot_LE',
'cupo_max_LE',
'cupo_prom_LE',
'cupo_tot_ME',
'cupo_max_ME',
'cupo_prom_ME',
'cupo_tot_SO',
'cupo_max_SO',
'cupo_prom_SO',
'cupo_tot_CC',
'cupo_max_CC',
'cupo_prom_CC',
'cupo_tot_TC_X',
'cupo_max_TC_X',
'cupo_prom_TC_X',
'cupo_tot_TC_T',
'cupo_max_TC_T',
'cupo_prom_TC_T',
'cupo_tot_tot',
'cupo_max_tot',
'cupo_prom_tot',
'cuota_tot_LB',
'cuota_max_LB',
'cuota_prom_LB',
'cuota_tot_TC',
'cuota_max_TC',
'cuota_prom_TC',
'cuota_tot_NV',
'cuota_max_NV',
'cuota_prom_NV',
'cuota_tot_CH',
'cuota_max_CH',
'cuota_prom_CH',
'cuota_tot_CE',
'cuota_max_CE',
'cuota_prom_CE',
'cuota_tot_LE',
'cuota_max_LE',
'cuota_prom_LE',
'cuota_tot_ME',
'cuota_max_ME',
'cuota_prom_ME',
'cuota_tot_SO',
'cuota_max_SO',
'cuota_prom_SO',
'cuota_tot_CC',
'cuota_max_CC',
'cuota_prom_CC',
'cuota_tot_TC_X',
'cuota_max_TC_X',
'cuota_prom_TC_X',
'cuota_tot_TC_T',
'cuota_max_TC_T',
'cuota_prom_TC_T',
'cuota_tot_tot',
'cuota_max_tot',
'cuota_prom_tot']

In [60]:
#Funcion que vuelve la variable en salarios minimos mensuales vigentes
def to_smlv(base,variable):
    
    base[variable+'_SMLV']=base[variable]/base['SMLV']
    del base[variable]
    
    
for var in variables_saldo_cupo_cuota:
    gc.collect()
    to_smlv(data_act_cli_fec,var)   

### Creación varibles comparación de mora mes anterior was_is

In [61]:
mora=data_act_cli_fec[['mora_maxLB','mora_maxTC','mora_maxNV','mora_maxCH','mora_maxCE','mora_maxLE','mora_maxME' ,'mora_maxSO','mora_maxCC','mora_maxTC_X','mora_maxTC_T','mora_max_tot']]

In [62]:
mora=mora.reset_index()

In [63]:
mora['llave1']=np.where(mora['id_cliente']==mora['id_cliente'].shift(1),1,0)

In [64]:
#Funcion que contruye las variables bucket por altura de mora
def altura_mora(cartera):
    mora['altura_mora_'+cartera]=np.where(mora['mora_max'+cartera]==0,0,
    np.where((mora['mora_max'+cartera]>=1) & (mora['mora_max'+cartera]<=30),1,
    np.where((mora['mora_max'+cartera]>=31) & (mora['mora_max'+cartera]<=60),2,
    np.where((mora['mora_max'+cartera]>=61) & (mora['mora_max'+cartera]<=90),3,
    np.where((mora['mora_max'+cartera]>=91) & (mora['mora_max'+cartera]<=120),4,
    np.where((mora['mora_max'+cartera]>=121) ,5,np.nan))))))

cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T','_tot']
for car in cartera3:
    altura_mora(car)   

In [65]:
#Funcion que contruye las variables lag

def var_lag(var,lag,lag1):
    mora['llave'+lag]=np.where(mora['id_cliente']==mora['id_cliente'].shift(lag1),1,0)
    mora[var+'_lag'+lag]=np.where(mora['llave'+lag]==1,mora[var].shift(lag1),np.nan)

cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T','_tot']
rezagos=['1','2','3','4','5','6']
for car in cartera3:
    for lag in rezagos:
        var='altura_mora_'+car
        lag1=int(lag)
        var_lag(var,lag,lag1)

        
for car in cartera3:
    for lag in rezagos:
        var='mora_max'+car
        lag1=int(lag)
        var_lag(var,lag,lag1)        

In [66]:
# Construcción variables was_is comparacion de la mora en dos puntos de observacion

In [67]:
cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T','_tot']
def was_is(cartera):   
    mora['was_is_'+cartera]=np.where((mora['altura_mora_'+cartera+'_lag1']==mora['altura_mora_'+cartera]),'mantiene',
                                             np.where((mora['altura_mora_'+cartera+'_lag1']>mora['altura_mora_'+cartera]) & (mora['altura_mora_'+cartera]>0),'devuelve',
                                             np.where((mora['altura_mora_'+cartera+'_lag1']>mora['altura_mora_'+cartera]) & (mora['altura_mora_'+cartera]==0),'normaliza',
                                             np.where((mora['altura_mora_'+cartera+'_lag1']<mora['altura_mora_'+cartera]) ,'rueda','NA'))))

for car in cartera3:
    was_is(car)

In [88]:
import gc
gc.collect()

40

In [76]:
mora_cols = pd.Series(mora.columns)[~pd.Series(mora.columns).isin(['id_cliente','periodo','was_is_LB','was_is_TC','was_is_NV','was_is_CH','was_is_CE','was_is_LE','was_is_ME' ,'was_is_SO','was_is_CC','was_is_TC_X','was_is_TC_T','was_is__tot'])]

In [77]:
mora.drop(columns=mora_cols,inplace=True)

In [78]:
mora.rename(columns={'was_is__tot':'was_is_tot'},inplace=True)

In [84]:
mora.set_index(['id_cliente','periodo'], inplace=True)

In [83]:
mora.head()

was_is_LB was_is_TC was_is_NV was_is_CH was_is_CE  \
id_cliente         periodo                                                     
101052294884460201 201904         NA        NA        NA        NA        NA   
                   201905         NA  mantiene        NA        NA        NA   
                   201906         NA  mantiene        NA        NA        NA   
                   201907         NA  mantiene        NA        NA        NA   
                   201908         NA  mantiene        NA        NA        NA   

                           was_is_LE was_is_ME was_is_SO was_is_CC  \
id_cliente         periodo                                           
101052294884460201 201904         NA        NA        NA        NA   
                   201905         NA        NA        NA        NA   
                   201906         NA        NA        NA        NA   
                   201907         NA        NA        NA        NA   
                   201908         NA        NA        NA        NA   

                           was_is_TC_X was_is_TC_T was_is_tot  
id_cliente         periodo                                     
101052294884460201 201904           NA          NA         NA  
                   201905     mantiene          NA   mantiene  
                   201906     mantiene          NA   mantiene  
                   201907     mantiene          NA   mantiene  
                   201908     mantiene          NA   mantiene

In [85]:
data_act_cli_fec=data_act_cli_fec.join(mora,how='left')

In [86]:
data_act_cli_fec.head()

SMLV  fecha_activo  num_aper_ultim_1meses_LB  \
id_cliente         periodo                                                     
101052294884460201 201904   828116.0        201904                       0.0   
                   201905   828116.0        201905                       0.0   
                   201906   828116.0        201906                       0.0   
                   201907   828116.0        201907                       0.0   
                   201908   828116.0        201908                       0.0   

                            num_aper_ultim_1meses_TC  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_NV  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_CH  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_CE  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_LE  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_ME  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_SO  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_1meses_CC  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                        0.0   
                   201908                        0.0   

                            num_aper_ultim_3meses_LB  \
id_cliente         periodo                             
101052294884460201 201904                        0.0   
                   201905                        0.0   
                   201906                        0.0   
                   201907                      

In [87]:
data_act_cli_fec.count()/len(data_act_cli_fec)

SMLV                           1.000000e+00
fecha_activo                   1.000000e+00
num_aper_ultim_1meses_LB       1.000000e+00
num_aper_ultim_1meses_TC       1.000000e+00
num_aper_ultim_1meses_NV       1.000000e+00
num_aper_ultim_1meses_CH       1.000000e+00
num_aper_ultim_1meses_CE       1.000000e+00
num_aper_ultim_1meses_LE       1.000000e+00
num_aper_ultim_1meses_ME       1.000000e+00
num_aper_ultim_1meses_SO       1.000000e+00
num_aper_ultim_1meses_CC       1.000000e+00
num_aper_ultim_3meses_LB       1.000000e+00
num_aper_ultim_3meses_TC       1.000000e+00
num_aper_ultim_3meses_NV       1.000000e+00
num_aper_ultim_3meses_CH       1.000000e+00
num_aper_ultim_3meses_CE       1.000000e+00
num_aper_ultim_3meses_LE       1.000000e+00
num_aper_ultim_3meses_ME       1.000000e+00
num_aper_ultim_3meses_SO       1.000000e+00
num_aper_ultim_3meses_CC       1.000000e+00
num_aper_ultim_6meses_LB       1.000000e+00
num_aper_ultim_6meses_TC       1.000000e+00
num_aper_ultim_6meses_NV       1

### Se exporta la base

In [89]:
import gc

gc.collect()

20

In [90]:
data_act_cli_fec.reset_index(inplace=True)

In [91]:
data_act.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_activos_201801_202006",engine='pyarrow', index=False)

In [92]:

data_act_cli_fec.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_activos_201801_202006_cli_fec",engine='pyarrow', index=False)

In [93]:
data_act_cli_fec.head()

,id_cliente,periodo,SMLV,fecha_activo,num_aper_ultim_1meses_LB,num_aper_ultim_1meses_TC,num_aper_ultim_1meses_NV,num_aper_ultim_1meses_CH,num_aper_ultim_1meses_CE,num_aper_ultim_1meses_LE,num_aper_ultim_1meses_ME,num_aper_ultim_1meses_SO,num_aper_ultim_1meses_CC,num_aper_ultim_3meses_LB,num_aper_ultim_3meses_TC,num_aper_ultim_3meses_NV,num_aper_ultim_3meses_CH,num_aper_ultim_3meses_CE,num_aper_ultim_3meses_LE,num_aper_ultim_3meses_ME,num_aper_ultim_3meses_SO,num_aper_ultim_3meses_CC,num_aper_ultim_6meses_LB,num_aper_ultim_6meses_TC,num_aper_ultim_6meses_NV,num_aper_ultim_6meses_CH,num_aper_ultim_6meses_CE,num_aper_ultim_6meses_LE,num_aper_ultim_6meses_ME,num_aper_ultim_6meses_SO,num_aper_ultim_6meses_CC,num_aper_ultim_12meses_LB,num_aper_ultim_12meses_TC,num_aper_ultim_12meses_NV,num_aper_ultim_12meses_CH,num_aper_ultim_12meses_CE,num_aper_ultim_12meses_LE,num_aper_ultim_12meses_ME,num_aper_ultim_12meses_SO,num_aper_ultim_12meses_CC,num_aper_ultim_1meses_TC_X,num_aper_ultim_1meses_TC_T,num_aper_ultim_3meses_TC_X,num_aper_ultim_3meses_TC_T,num_aper_ultim_6meses_TC_X,num_aper_ultim_6meses_TC_T,num_aper_ultim_12meses_TC_X,num_aper_ultim_12meses_TC_T,num_aper_ultim_1meses_tot,num_aper_ultim_3meses_tot,num_aper_ultim_6meses_tot,num_aper_ultim_12meses_tot,meses_ultim_aper_LB,meses_ultim_aper_TC,meses_ultim_aper_NV,meses_ultim_aper_CH,meses_ultim_aper_CE,meses_ultim_aper_LE,meses_ultim_aper_ME,meses_ultim_aper_SO,meses_ultim_aper_CC,meses_ultim_aper_TC_X,meses_ultim_aper_TC_T,meses_ultim_aper_tot,antiguedad_tot_conlibranza,antiguedad_tot_sinlibranza,antiguedad_TDC_T,antiguedad_TDC_X,num_obl_vig_LB,num_obl_vig_TC,num_obl_vig_NV,num_obl_vig_CH,num_obl_vig_CE,num_obl_vig_LE,num_obl_vig_ME,num_obl_vig_SO,num_obl_vig_CC,num_obl_vig_TC_X,num_obl_vig_TC_T,num_obl_vig_tot,num_obl_aldia_LB,porc_obl_aldia_LB,num_obl_mora1_30_LB,porc_obl_mora1_30_LB,num_obl_mora31_60_LB,porc_obl_mora31_60_LB,num_obl_mora61_90_LB,porc_obl_mora61_90_LB,num_obl_mora91_120_LB,porc_obl_mora91_120_LB,num_obl_mora121_mas_LB,porc_obl_mora121_mas_LB,num_obl_aldia_TC,porc_obl_aldia_TC,num_obl_mora1_30_TC,porc_obl_mora1_30_TC,num_obl_mora31_60_TC,porc_obl_mora31_60_TC,num_obl_mora61_90_TC,porc_obl_mora61_90_TC,num_obl_mora91_120_TC,porc_obl_mora91_120_TC,num_obl_mora121_mas_TC,porc_obl_mora121_mas_TC,num_obl_aldia_NV,porc_obl_aldia_NV,num_obl_mora1_30_NV,porc_obl_mora1_30_NV,num_obl_mora31_60_NV,porc_obl_mora31_60_NV,num_obl_mora61_90_NV,porc_obl_mora61_90_NV,num_obl_mora91_120_NV,porc_obl_mora91_120_NV,num_obl_mora121_mas_NV,porc_obl_mora121_mas_NV,num_obl_aldia_CH,porc_obl_aldia_CH,num_obl_mora1_30_CH,porc_obl_mora1_30_CH,num_obl_mora31_60_CH,porc_obl_mora31_60_CH,num_obl_mora61_90_CH,porc_obl_mora61_90_CH,num_obl_mora91_120_CH,porc_obl_mora91_120_CH,num_obl_mora121_mas_CH,porc_obl_mora121_mas_CH,num_obl_aldia_CE,porc_obl_aldia_CE,num_obl_mora1_30_CE,porc_obl_mora1_30_CE,num_obl_mora31_60_CE,porc_obl_mora31_60_CE,num_obl_mora61_90_CE,porc_obl_mora61_90_CE,num_obl_mora91_120_CE,porc_obl_mora91_120_CE,num_obl_mora121_mas_CE,porc_obl_mora121_mas_CE,num_obl_aldia_LE,porc_obl_aldia_LE,num_obl_mora1_30_LE,porc_obl_mora1_30_LE,num_obl_mora31_60_LE,porc_obl_mora31_60_LE,num_obl_mora61_90_LE,porc_obl_mora61_90_LE,num_obl_mora91_120_LE,porc_obl_mora91_120_LE,num_obl_mora121_mas_LE,porc_obl_mora121_mas_LE,num_obl_aldia_ME,porc_obl_aldia_ME,num_obl_mora1_30_ME,porc_obl_mora1_30_ME,num_obl_mora31_60_ME,porc_obl_mora31_60_ME,num_obl_mora61_90_ME,porc_obl_mora61_90_ME,num_obl_mora91_120_ME,porc_obl_mora91_120_ME,num_obl_mora121_mas_ME,porc_obl_mora121_mas_ME,num_obl_aldia_SO,porc_obl_aldia_SO,num_obl_mora1_30_SO,porc_obl_mora1_30_SO,num_obl_mora31_60_SO,porc_obl_mora31_60_SO,num_obl_mora61_90_SO,porc_obl_mora61_90_SO,num_obl_mora91_120_SO,porc_obl_mora91_120_SO,num_obl_mora121_mas_SO,porc_obl_mora121_mas_SO,num_obl_aldia_CC,porc_obl_aldia_CC,num_obl_mora1_30_CC,porc_obl_mora1_30_CC,num_obl_mora31_60_CC,porc_obl_mora31_60_CC,num_obl_mora61_90_CC,porc_obl_mora61_90_CC,num_obl_mora91_120

In [ ]:
no Correr ###

## Las variables a continuacion se contruyen en Spark ya que la memoria no es suficinte

## Variables Ultimos x meses

In [ ]:
tendencias=activos_201809_201907_cli_fec[['mora_maxLB','mora_maxTC','mora_maxNV','mora_maxCH','mora_maxCE','mora_maxLE','mora_maxME' ,'mora_maxSO','mora_maxCC','mora_maxTC_X','mora_maxTC_T','mora_max_tot',
                                        'utilizacion_prom_TC','utilizacion_prom_TC_X','utilizacion_prom_TC_T','cupo_tot_LE_SMLV', 'cuota_tot_TC_X_SMLV', 'saldo_tot_LB_SMLV', 'cupo_tot_LB_SMLV', 'cuota_tot_tot_SMLV', 'cupo_tot_tot_SMLV', 'cupo_tot_TC_X_SMLV', 'cupo_tot_TC_SMLV', 'cuota_tot_CH_SMLV', 'cupo_tot_SO_SMLV', 'cupo_tot_CH_SMLV', 'cupo_tot_ME_SMLV', 'saldo_tot_TC_X_SMLV', 'saldo_tot_NV_SMLV', 'saldo_tot_LE_SMLV', 'cupo_tot_NV_SMLV', 'saldo_tot_TC_T_SMLV', 'cuota_tot_CE_SMLV', 'cuota_tot_LE_SMLV', 'cuota_tot_SO_SMLV', 'cuota_tot_TC_T_SMLV', 'saldo_tot_tot_SMLV', 'cuota_tot_LB_SMLV', 'cupo_tot_TC_T_SMLV', 'cuota_tot_CC_SMLV', 'cupo_tot_CE_SMLV', 'cuota_tot_ME_SMLV', 'saldo_tot_CC_SMLV', 'cuota_tot_TC_SMLV', 'cuota_tot_NV_SMLV', 'saldo_tot_CH_SMLV', 'saldo_tot_SO_SMLV', 'saldo_tot_TC_SMLV', 'saldo_tot_CE_SMLV', 'cupo_tot_CC_SMLV', 'saldo_tot_ME_SMLV']]

In [ ]:
##Variables llave para asegurar que se comparan registros del mismo cliente
activos_201809_201907_cli_fec=activos_201809_201907_cli_fec.reset_index()
activos_201809_201907_cli_fec['llave1']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(1)==activos_201809_201907_cli_fec['id_cliente']),1,0)
activos_201809_201907_cli_fec['llave2']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(2)==activos_201809_201907_cli_fec['id_cliente']),1,0)
activos_201809_201907_cli_fec['llave3']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(3)==activos_201809_201907_cli_fec['id_cliente']),1,0)
activos_201809_201907_cli_fec['llave4']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(4)==activos_201809_201907_cli_fec['id_cliente']),1,0)
activos_201809_201907_cli_fec['llave5']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(5)==activos_201809_201907_cli_fec['id_cliente']),1,0)
activos_201809_201907_cli_fec['llave6']=np.where((activos_201809_201907_cli_fec['id_cliente'].shift(6)==activos_201809_201907_cli_fec['id_cliente']),1,0)


### Variables Mora maxima últimos 3,6 meses

In [ ]:
#Funcion que contruye las variables lag

def var_lag(var):
    activos_201809_201907_cli_fec[var+'_lag1']=activos_201809_201907_cli_fec[var].shift(1)
    activos_201809_201907_cli_fec[var+'_lag2']=activos_201809_201907_cli_fec[var].shift(2)
    activos_201809_201907_cli_fec[var+'_lag3']=activos_201809_201907_cli_fec[var].shift(3)
    activos_201809_201907_cli_fec[var+'_lag4']=activos_201809_201907_cli_fec[var].shift(4)
    activos_201809_201907_cli_fec[var+'_lag5']=activos_201809_201907_cli_fec[var].shift(5)
    activos_201809_201907_cli_fec[var+'_lag6']=activos_201809_201907_cli_fec[var].shift(6)

cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T']
for car in cartera3:
    var='mora_max_'+cartera
    var_lag(var)

for car in cartera3:
    var='saldo_tot_'+cartera
    var_lag(var)
    
for car in cartera3:
    var='cuota_tot_'+cartera
    var_lag(var)
    
for car in cartera3:
    var='cupo_tot_'+cartera
    var_lag(var)

for car in cartera3:
    var='utilizacion_prom_'+cartera
    var_lag(var)

### Variables Mora maxima últimos 3,6 meses

In [ ]:
#Función que calcula el mora maxima de los ultimos x meses por cartera
def mora_max_ult6(cartera):
    activos_201809_201907_cli_fec['mora_max_6m_'+cartera]=np.where((activos_201809_201907_cli_fec['llave1']==1) & (activos_201809_201907_cli_fec['llave2']==1) & (activos_201809_201907_cli_fec['llave3']==1) & (activos_201809_201907_cli_fec['llave4']==1) & (activos_201809_201907_cli_fec['llave5']==1) & (activos_201809_201907_cli_fec['llave6']==1),activos_201809_201907_cli_fec[['mora_lag1','mora_lag2','mora_lag3','mora_lag4','mora_lag5','mora_lag6']].max(axis=1),np.nan)

def mora_max_ult6(cartera):
    activos_201809_201907_cli_fec['mora_max_3m_'+cartera]=np.where((activos_201809_201907_cli_fec['llave1']==1) & (activos_201809_201907_cli_fec['llave2']==1) & (activos_201809_201907_cli_fec['llave3']==1) ),activos_201809_201907_cli_fec[['mora_lag1','mora_lag2','mora_lag3']].max(axis=1),np.nan)

cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T']
for car in cartera3:
    mora_max_ult(car)
    mora_max_ult(car)

### Variables saldo,cupo,cuota y utilizacion promedio ultimos 3 y 6 meses

In [ ]:
#Función que calcula el promedio de los ultimos x meses 
def prom_ult6(var,meses,cartera):
    activos_201809_201907_cli_fec[var+'prom_6m']=np.where((activos_201809_201907_cli_fec['llave1']==1) & (activos_201809_201907_cli_fec['llave2']==1) & (activos_201809_201907_cli_fec['llave3']==1) & (activos_201809_201907_cli_fec['llave4']==1) & (activos_201809_201907_cli_fec['llave5']==1) & (activos_201809_201907_cli_fec['llave6']==1),activos_201809_201907_cli_fec[[var+'_lag1',var+'_lag2',var+'_lag3',var+'_lag4',var+'_lag5',var+'_lag6']].mean(axis=1),np.nan)

def prom_ult3(var,meses,cartera):
    activos_201809_201907_cli_fec[var+'prom_3m']=np.where((activos_201809_201907_cli_fec['llave1']==1) & (activos_201809_201907_cli_fec['llave2']==1) & (activos_201809_201907_cli_fec['llave3']==1) ),activos_201809_201907_cli_fec[[var+'_lag1',var+'_lag2',var+'_lag3']].mean(axis=1),np.nan)


cartera3=['LB','TC','NV','CH','CE','LE','ME' ,'SO','CC','TC_X','TC_T']


for car in cartera3:
    var='saldo_tot_'+car
    prom_ult6(var,car)
    prom_ult3(var,car)
        
for car in cartera3:
    var='cupo_tot_'+car
    prom_ult6(var,car)
    prom_ult3(var,car) 
    
for car in cartera3:
    var='cuota_tot_'+car
    prom_ult6(var,car)
    prom_ult3(var,car)

for car in cartera3:
    var='utilizacion_prom_'+car
    prom_ult6(var,car)
    prom_ult3(var,car)